In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as pyplot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
data = pd.read_csv('./data.csv')

In [3]:
data.shape

(1304543, 23)

In [7]:
data.columns =['MeterNo','AccountName','AccountType','BillCycle','BillCycleID','Date','DateStr',
               'Event','ExceptionType','ExceptionTypeDesc',
               'MeterType','MeterReadType','MeterReadTypeDesc','PremiseName','ServicePt',
               'UOM','UOMName','TimeStamp','CurrBillCycle',
               'DateId','Value1','Value2','Record_count']

In [8]:
data.head(5)

,MeterNo,AccountName,AccountType,BillCycle,BillCycleID,Date,DateStr,Event,ExceptionType,ExceptionTypeDesc,...,PremiseName,ServicePt,UOM,UOMName,TimeStamp,CurrBillCycle,DateId,Value1,Value2,Record_count
0,1679,Thane,RESIDENTIAL,BILL CYCLE 10,BC10,2017-02-03,20170203,NONE,NONE,NONE,...,33rd Street,SP33,kw,kilowatts,21:45:00,0,34,0.2740,NaN,1
1,1679,Thane,RESIDENTIAL,BILL CYCLE 10,BC10,2017-03-15,20170315,NONE,NONE,NONE,...,33rd Street,SP33,kw,kilowatts,22:15:00,0,74,0.6577,NaN,1
2,1679,Thane,RESIDENTIAL,BILL CYCLE 10,BC10,2017-01-19,20170119,NONE,NONE,NONE,...,33rd Street,SP33,kw,kilowatts,22:00:00,0,19,0.9483,NaN,1
3,1679,Thane,RESIDENTIAL,BILL CYCLE 10,BC10,2017-03-21,20170321,NONE,NONE,NONE,...,33rd Street,SP33,kw,kilowatts,21:30:00,0,80,0.8360,NaN,1
4,1679,Thane,RESIDENTIAL,BILL CYCLE 10,BC10,2017-03-18,20170318,NONE,NONE,NONE,...,33rd Street,SP33,kw,kilowatts,22:45:00,0,77,0.9974,NaN,1


In [9]:
data[data['ExceptionType'] != 'NONE']
data[data['ExceptionType'] != 'NONE'].count()

MeterNo              242
AccountName          242
AccountType          242
BillCycle            242
BillCycleID          242
Date                 242
DateStr              242
Event                242
ExceptionType        242
ExceptionTypeDesc    242
MeterType            242
MeterReadType        242
MeterReadTypeDesc    242
PremiseName          242
ServicePt            242
UOM                  242
UOMName              242
TimeStamp            242
CurrBillCycle        242
DateId               242
Value1               242
Value2                 0
Record_count         242
dtype: int64

In [11]:
print(data['AccountType'].unique())
print(data['BillCycleID'].unique())
print(data['Event'].unique())
print(data['ExceptionType'].unique())
print(data['MeterType'].unique())
print(data['MeterReadTypeDesc'].unique())

['RESIDENTIAL' 'COMMERCIAL' 'INDUSTRIAL']
['BC10' 'BC16' 'BC5' 'BC15' 'BC4' 'BC8' 'BC7' 'BC22' 'BC21' 'BC3' 'BC20'
 'BC2' 'BC14' 'BC19' 'BC18' 'BC13' 'BC1' 'BC17' 'BC12' 'BC11' 'BC9' 'BC6']
['NONE' 'Opened' 'Changed' 'LossDetected' 'Reset' 'Error' 'Cleared'
 'MinLimitReached' 'Stopped' 'Failed']
['NONE' 'Security' 'Configuration Failure' 'Power Outage'
 'Inaccurate Reading/Data Collection Failure' 'Firmware Failure'
 'Service/Meter Type Mismatch' 'Battery Failure' 'Communication Failure'
 'Meter Clock Failure']
['E']
['ACTUAL' 'MISSING' 'ESTIMATED']


In [6]:
data.dtypes

MeterNo                int64
AccountName           object
AccountType           object
BillCycle             object
BillCycleID           object
Date                  object
DateStr                int64
Event                 object
ExceptionType         object
ExceptionTypeDesc     object
MeterType             object
MeterReadType         object
MeterReadTypeDesc     object
ServicePt             object
UOM                   object
UOMName               object
TimeStamp             object
CurrBillCycle          int64
PremiseName           object
Latitude             float64
Longitude            float64
DateId                 int64
Value1               float64
Value2               float64
Record_count           int64
dtype: object

In [ ]:
data['MeterNo'] = data['MeterNo'].astype('str')
data['AccountType'] = data['AccountType'].astype('category')
data['BillCycleID'] = data['BillCycleID'].astype('category')
data['Event'] = data['Event'].astype('category')
data['ExceptionTypeDesc'] = data['ExceptionTypeDesc'].astype('category')
data['MeterReadTypeDesc'] = data['MeterReadTypeDesc'].astype('category')